# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 6: Building an interactive dashboard using Plotly and Dash


## Objectives


In this lab, we will build a dashboard using pandas dash package and plot charts using Plotly..

*   Build an interactive dashboard app using pandas dash.
*   Plot interactive charts in the dashboard using Plotly.


***

Install the necessary packages

In [1]:
!pip install pandas dash
!pip install wget

     |████████████████████████████████| 7.3 MB 15.5 MB/s eta 0:00:01
     |████████████████████████████████| 23.9 MB 47.4 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 6.7 MB/s  eta 0:00:01
     |████████████████████████████████| 288 kB 38.3 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 42.4 MB/s eta 0:00:01
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3804 sha256=f2c0d5a3d56facdd9ef94a0a851e197faf26425ff9066508cfca674134fbe314
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/52/e4/f3/16724791571a955a46d54650510c98c04ab7d339626aee27cc
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4069 sha256=626953cf36a2a743dd8757f8e5948bd70af5a0147a5257369c8d76e45095ec14
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/73/d8/8d/92f612c03c895f19bcc56a6c54be7bb41aaa698012a5624f60
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl 

Download the dataset

In [5]:
!wget -O spacex_launch_dash.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv

Inspect the dataset

In [6]:
import pandas as pd
spacex_data = pd.read_csv('spacex_launch_dash.csv')
spacex_data.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


Install ngrok for running the app in a separate tab in your browser

In [8]:
### Install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

### Run ngrok to tunnel Dash app port 8050 to the outside world. 
### This command runs in the background.
get_ipython().system_raw('./ngrok http 8050 &')

### Get the public URL where you can access the Dash app. Copy this URL.
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Build the dash app and save it as `spacex_dash_app.py`

In [9]:
%%writefile spacex_dash_app.py
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                html.Label('Dropdown'),
                                dcc.Dropdown(id='site-dropdown',
                                    options=[
                                             {'label':'All Sites', 'value':'All Sites'},
                                             {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                             {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                             {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                             {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                    ],
                                    value='All Sites',
                                    placeholder='Select a Launch Site here',
                                    searchable=True
                                ),
                                html.Br(),

                                # Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000,
                                                marks={i: '{} (Kg)'.format(i) for i in range(0, 10001, 1000)},
                                                value=[min_payload, max_payload]),

                                # Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))
def get_pie(site_sel):
  if site_sel == 'All Sites':
    piedata = spacex_df[['Launch Site', 'class']]
    piedat = piedata.groupby(['Launch Site']).sum()
    pie_fig = px.pie(piedat, values="class", names=piedat.index, title='Total successful launches by site')
    return pie_fig
  else:
    piedata = spacex_df[spacex_df['Launch Site']==site_sel]
    piedat = piedata['class'].value_counts()
    pie_fig = px.pie(piedat, names=piedat.index, values=piedat.values, title='Total successful launches for site {}'.format(site_sel))
    return pie_fig

# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
               [Input(component_id='site-dropdown', component_property='value'),
                Input(component_id='payload-slider', component_property='value')])
def get_sct(sel_site, payload):
  if sel_site != 'All Sites':
    sctdata = spacex_df[['Launch Site', 'class', 'Payload Mass (kg)', 'Booster Version Category']]
    sctdat1 = sctdata[sctdata['Launch Site']==sel_site]
    sctdat = sctdat1[(sctdat1['Payload Mass (kg)'] >= payload[0]) & (sctdat1['Payload Mass (kg)'] <= payload[1])]
    sct_fig = px.scatter(sctdat, x="Payload Mass (kg)", y="class", color="Booster Version Category")
    return sct_fig
  else:
    sctdata = spacex_df[['Launch Site', 'class', 'Payload Mass (kg)', 'Booster Version Category']]
    sctdat = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload[0]) & (spacex_df['Payload Mass (kg)'] <= payload[1])]
    sct_fig = px.scatter(sctdat, x="Payload Mass (kg)", y="class", color="Booster Version Category")
    return sct_fig
    
# Run the app
if __name__ == '__main__':
    app.run_server()

Overwriting spacex_dash_app.py


Start running the app, then click on the link obtained from ngrok

In [9]:
!python spacex_dash_app.py